In [1]:
!pip install --upgrade scikit-learn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold, KFold,cross_val_predict,cross_val_score,cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgbm
import xgboost as xgb
import pickle
# from google.colab import files
import category_encoders as ce
from catboost import CatBoostClassifier, Pool
from sklearn.feature_selection import SelectKBest, f_classif,chi2,SelectPercentile
from scipy.stats import rankdata
from sklearn.mixture import GaussianMixture 
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve

import string
import re

In [3]:
df_all = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
df_submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [4]:
X = df_all.drop(labels=['id','target'],axis=1,inplace=False).copy()
y = df_all['target'].values
X_test = df_test.drop(labels=['id'],axis=1,inplace=False).copy()
random_seed = 2001

encoder = OrdinalEncoder()
all_encoded = encoder.fit_transform(X.append(X_test))
X = all_encoded[0:len(X)]
X_test = all_encoded[len(X):]

In [5]:
# params = {
#           'bootstrap':True,
#           'max_depth':30,
#           'max_features':'auto'  ,
#           'min_samples_leaf' :10,
#           'min_samples_split':5,
#           'n_estimators':1560
#           }

params = {'max_depth': 22, 
          'n_estimators': 1620, 
          'max_features': 'sqrt', 
          'min_samples_split': 2, 
          'bootstrap': False, 
          'min_samples_leaf': 5}

In [6]:
name = 'random_forest'
k=5
seed_list=[2001,2002,2003]
kf = StratifiedKFold(n_splits=k,shuffle=True,random_state=random_seed)
oof = np.zeros((len(df_all),9))
test_preds_list = []
score_list = []
fold=1
  
splits = list(kf.split(X,y))
fold = 1
for train_idx, val_idx in splits:
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    val_preds_list = []

    for seed in seed_list:

        # fit and run model

        base_model = RandomForestClassifier(random_state=seed)
        model = CalibratedClassifierCV(base_model, method='isotonic', cv=k)

        model.fit(X_train,y=y_train)


        val_preds_list.append(model.predict_proba(X_val))
        test_preds_list.append(model.predict_proba(X_test))

    oof[val_idx] = np.mean(val_preds_list,axis=0)
    score = log_loss(y_val, oof[val_idx])
    print(f"fold: {fold},log_loss: {score}")
    score_list.append(score)
    # print(f"fold: {fold}, class0 tr %: {y_train.value_counts()[0]/len(y_train)}, class0 val %: {y_val.value_counts()[0]/len(y_val)} ")
    fold +=1

cv_logloss = np.mean(score_list)
print(f"{name} ,log_loss: {cv_logloss}")

preds= np.mean(test_preds_list,axis=0)


file_name_oof = "rfc_3seeds_oof.txt"
file_name_test = "rfc_3seeds_test.csv"
with open(file_name_oof, "wb") as fp:
      pickle.dump(oof, fp)

#files.download(file_name_oof)

df_submission[['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']] = preds
df_submission.to_csv(file_name_test,index=None)
#files.download(file_name_test) 

fold: 1,log_loss: 1.7626182788118505
fold: 2,log_loss: 1.7627925974953298
fold: 3,log_loss: 1.759443962149712
fold: 4,log_loss: 1.7616900211200333
fold: 5,log_loss: 1.7599808811239506
random_forest ,log_loss: 1.7613051481401754
